In [ ]:
# 查看当前挂载的数据集目录
!ls /home/mw/input/

In [ ]:
# 查看个人持久化工作区文件
!ls /home/mw/work/

In [ ]:
# 查看当前kernel下的package
!pip list --format=columns

In [ ]:
# 显示cell运行时长
%load_ext klab-autotime

In [5]:
# !pip install bayesian-optimization

In [8]:
import pandas as pd
import gc
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.metrics import confusion_matrix
%matplotlib inline
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
import lightgbm as lgb
from sklearn.metrics import fbeta_score
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold,StratifiedKFold

In [9]:
# df_train = pd.read_csv('/home/mw/input/tianchi6504/model_sample.csv')
df_train = pd.read_csv('/home/mw/work/features.csv',index_col =0)
df_train.reset_index()

df_train.head()

,user_id,y,x_001,x_002,x_003,x_004,x_005,x_006,x_007,x_008,...,x_184/x_179,x_185/x_180,x_189/x_188,x_191/x_190,x_193/x_192,x_195/x_194,x_197/x_196,x_199/x_198,x_196/x_188,x_192/x_188
0,A00002,0,0.0,32.0,0,0,0,0,0,0,...,0.25000,1000.000000,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0
1,A00005,0,0.0,29.0,0,0,0,0,0,0,...,-999.00000,-999.000000,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
2,A00006,0,0.0,31.0,0,0,0,0,0,0,...,0.62069,1606.939091,1.0,1.0,1.0,1.0,1.0,1.0,3.0,2.0
3,A00008,0,0.0,22.0,0,0,0,0,0,0,...,0.00000,1570.506667,1.0,1.0,1.0,1.0,1.0,1.0,1.5,1.5
4,A00009,0,0.0,31.0,0,0,0,0,0,0,...,-999.00000,-999.000000,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


In [13]:
no_features = ['y','user_id']
seed =2021
features = [col for col in df_train.columns if col not in no_features]
ycol = 'y'
feature_names = features

In [14]:
from sklearn.model_selection import cross_val_score

"""定义优化函数"""
def rf_cv_lgb(num_leaves, max_depth, bagging_fraction, feature_fraction, bagging_freq, min_data_in_leaf, 
              min_child_weight, min_split_gain, reg_lambda, reg_alpha):
    # 建立模型
    model_lgb = lgb.LGBMClassifier(boosting_type='gbdt', bjective='binary', metric='auc',
                                   learning_rate=0.1, n_estimators=5000,
                                   num_leaves=int(num_leaves), max_depth=int(max_depth), 
                                   bagging_fraction=round(bagging_fraction, 2), feature_fraction=round(feature_fraction, 2),
                                   bagging_freq=int(bagging_freq), min_data_in_leaf=int(min_data_in_leaf),
                                   min_child_weight=min_child_weight, min_split_gain=min_split_gain,
                                   reg_lambda=reg_lambda, reg_alpha=reg_alpha,
                                   n_jobs= 8
                                  )
    
    val = cross_val_score(model_lgb,df_train[feature_names],df_train[ycol], cv=5, scoring='roc_auc').mean()
    
    return val

In [15]:
from bayes_opt import BayesianOptimization
"""定义优化参数"""
bayes_lgb = BayesianOptimization(
    rf_cv_lgb, 
    {
        'num_leaves':(10, 200),
        'max_depth':(3, 20),
        'bagging_fraction':(0.5, 1.0),
        'feature_fraction':(0.5, 1.0),
        'bagging_freq':(0, 100),
        'min_data_in_leaf':(10,100),
        'min_child_weight':(0, 10),
        'min_split_gain':(0.0, 1.0),
        'reg_alpha':(0.0, 10),
        'reg_lambda':(0.0, 10),
    }
)

"""开始优化"""
bayes_lgb.maximize(n_iter=10)

|   iter    |  target   | baggin... | baggin... | featur... | max_depth | min_ch... | min_da... | min_sp... | num_le... | reg_alpha | reg_la... |
-------------------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.7905   |  0.7123   |  54.88    |  0.7242   |  6.232    |  2.757    |  88.13    |  0.7884   |  179.1    |  8.524    |  0.6577   |
|  2        |  0.7876   |  0.6279   |  96.56    |  0.8569   |  14.81    |  2.636    |  82.41    |  0.3409   |  125.1    |  4.818    |  1.978    |
|  3        |  0.7884   |  0.7398   |  57.8     |  0.9446   |  9.199    |  9.902    |  49.54    |  0.4175   |  187.4    |  4.75     |  8.719    |
|  4        |  0.7869   |  0.8944   |  79.46    |  0.9095   |  19.87    |  3.69     |  25.29    |  0.0838   |  49.5     |  9.605    |  9.67     |
|  5        |  0.7877   |  0.8228   |  35.27    |  0.7648   |  14.5     |  9.738    |  24.69    |  0.7303   |  181.2    |  2

In [17]:
bayes_lgb.max


{'target': 0.7927779337442866,
 'params': {'bagging_fraction': 0.8256838817068106,
  'bagging_freq': 12.897449911782854,
  'feature_fraction': 0.6799417915036878,
  'max_depth': 19.607609424498186,
  'min_child_weight': 7.989484773056548,
  'min_data_in_leaf': 92.87862688573793,
  'min_split_gain': 0.5653189848881633,
  'num_leaves': 178.8305250513094,
  'reg_alpha': 9.992751318598641,
  'reg_lambda': 1.2535560707642368}}